# Empirical Assignment (Q1)
## Economics 35310: Topics in Trade & Growth
### Author: Levi Crews

In this notebook I simulate an economy of 10,000 firms using the model from TKMD (2018). Firms are indexed from 1 to 10,000. Each set of eligible suppliers (max cardinality of 300) is drawn randomly from the set of firms whose indices precede that of the buyer. Each firm $j$ is characterized by
+ its set of eligible suppliers that satisfies the ordering ($Z_j$)
+ its core productivity level ($\phi_j$)
+ its vector of firm-pair-specific cost shifters ($\alpha_{kj}$)
+ its foreign input cost shifter ($\alpha_{Fj}$)
+ its foreign demand shifter ($\beta_{jF}$)

I normalize firms' labor productivity shifters, $\alpha_{Lj} = 1$, and firms' domestic final demand shifters, $\beta_{jH} = 1$. I parameterize firm characteristics in the following way:
+ $\alpha_{kj} = X_k \times Y_{kj}$ with $X_k \sim $ Beta(0.1, 0.9) and $Y_{kj} \sim $ log-normal($\Phi_{scale}^{\alpha_{dom}}, \Phi_{disp}^{\alpha, \beta}$) i.i.d.
+ $\alpha_{Fj} \sim $ log-normal($\Phi_{scale}^{\alpha_F}, \Phi_{disp}^{\alpha, \beta}$) i.i.d.
+ $\beta_{jF} \sim $ log-normal($\Phi_{scale}^{\beta_F}, \Phi_{disp}^{\alpha, \beta}$) i.i.d.

For the 8 parameters $\Phi$ I use the estimated values in Table 4 of TKMD (2018):

| $\Phi_{scale}^{\alpha_{dom}}$ | $\Phi_{scale}^{\alpha_F}$ | $\Phi_{scale}^{\beta_F}$ | $\Phi_{disp}^{\alpha, \beta}$ |
|--|--|--|--|
| -4.42 | -2.22 | -2.01 | 2.34 |

In this simulation I try to match the following moments of the Belgian data presented in Table 5 of TKMD (2018):
+ number of importing firms: 19,000
+ number of exporting firms: 12,000
+ labor share in cost: 0.17 / 0.34 / 0.54 (25/50/75 percentiles)
+ productivity: $\phi_j \sim $ log-normal (-1.52, 0.85)
+ total foreign input share: 0.24 / 0.39 / 0.55 (25/50/75 percentiles)
+ number of domestic suppliers: 19 / 33 / 55 (25/50/75 percentiles)
+ number of domestic customers: 2 / 9 / 34 (25/50/75 percentiles)

Note that I normalized $\frac{PE^{1/(\sigma-1)}}{\mu w} = 1$ to generate the productivity distribution. In addition, I calibrate $\sigma = 4$ and $\rho = 2$ following the baseline specification of TKMD (2018).

I will then (i) calculate the total foreign input share for each simulated firm and (ii) calculate the real wage change from going to autarky.

Finally, I will (iii) simulate a 10% increase in the import price and (iv) calculate the concomitant change in the real wage, holding the network fixed.

### Disclaimer

I woefully fail at matching the *cost shares* and the *quartiles of the customer distribution*. I would link to think that fiddling with my distribution parameters could rectify this issue. That said, I apologize if there is a bug in my code.

In [1]:
# Load packages 
using Distributions, StatsBase, Random, Statistics
using LinearAlgebra, SparseArrays, NLsolve, IterativeSolvers
using Plots
unicodeplots();

In [2]:
# Load user-defined functions
# Change path as needed
include("C:\\Users\\levic\\Dropbox\\University-of-Chicago\\Academics\\2-SecondYear\\Econ35310-TradeGrowth\\Empirical-Project\\Crews_Econ35310_SparseShares.jl");
include("C:\\Users\\levic\\Dropbox\\University-of-Chicago\\Academics\\2-SecondYear\\Econ35310-TradeGrowth\\Empirical-Project\\Crews_Econ35310_FixNetEquil.jl");

In [3]:
# Set seed
Random.seed!(4321);

### Initializing the Acyclic Network

I begin by generating an acyclic network. First, I index firms 1 to 10,000. Then, for each firm $j$ I draw the cardinality of its set of eligible domestic suppliers, $Z_j^D$ from a discretized log-normal(3.5, 0.8) distribution truncated at min$\{j,$ 300$\}$. With the cardinality of $Z_j^D$ in hand for each firm $j$, I then randomly select $|Z_j^D|$ indices $i$ such that $i < j$ for each firm $j$. After this, I randomly assign import/export decisions in order to match the number of importing and exporting firms from Table 5. This completes the network.

In [4]:
# Initialize network
num = 10000 # note: I was running out of memory with 100,000 firms, so I did 10,000
maxcard = 300
Nmu = 3.5
Nsigma = 0.8

N = zeros(num, 2)
N[:, 1] = collect(1:num)
for i = 1:maxcard
    temp = min(rand(LogNormal(Nmu, Nsigma)), i-1)
    N[i, 2] = ceil(Int64, temp)
end
temp = min.(rand(LogNormal(Nmu, Nsigma), num-maxcard), maxcard*ones(num-maxcard,1))
N[maxcard+1:end, 2] = ceil.(Int64, temp)
N = Array{Int64}(N);

In [5]:
# Randomly select sets of eligible suppliers
E = zeros(num, maxcard)
for j = 2:num
    temp = sample(N[1:j-1, 1], N[j,2]; replace=false, ordered=true)
    for i = 1:N[j, 2]
        E[j,i] = temp[i,1]
    end
end
E = Array{Int64}(E);

In [6]:
# Randomly assign import/export decisions
F = rand(num, 2)
for j = 1:num
    if F[j,1] < 0.19
        F[j,1] = 1 
    else 
        F[j,1] = 0
    end
    if F[j,2] < 0.12
        F[j,2] = 1 
    else 
        F[j,2] = 0
    end
end
F = Array{Int64}(F);

In [7]:
# Check if moments matched
println(sum(F[:,1])) # num. importers
println(sum(F[:,2])) # num. exporters

1877
1217


In [8]:
# Concatenate (N,E,F) to one network matrix
N = hcat(N, E, F);

In initializing the network, I made no effort to match the covariances between any two of (i) firm productivity, (ii) importing decision, (iii) exporting decision, (iv) cost shares, or (v) position in the network. In reality, I'd expect, for example, that more productive firms would be more likely to export. I abstract from these considerations for simplicity.

### Assigning Firm Characteristics

I now assign firm characteristics according to the distributions listed at the outset. Again, I do not consider any covariance among firm characteristics or between firm characteristics and the network.

In [9]:
# Initialize parameters
Phi_ADscale = -4.42
Phi_AFscale = -2.22
Phi_BFscale = -2.01
Phi_ABdisp = 2.34

# Draw firm characteristics
A = zeros(num, 8)
A[:,1] = rand(LogNormal(-1.52, 0.85), num) # \phi_j
A[:,2] = rand(Beta(0.1, 0.9), num) # X_j
A[:,3] = rand(LogNormal(Phi_ADscale, Phi_ABdisp), num) # Y_{kj} for all k
A[:,4] = rand(LogNormal(Phi_AFscale, Phi_ABdisp), num) # \alpha_{Fj}
A[:,5] = rand(LogNormal(Phi_BFscale, Phi_ABdisp), num) # \beta_{jF}

# Concatenate network and firm characteristics
N = hcat(N, A);

### Simulating the Economy

Recall the definition of a competitive equilibrium with a fixed network: Given foreign expenditure $E_f$, foreign price index $P_F$, and a set of prices set by foreign suppliers $\{p_{Fj}\}$, an equilibrium for the model with a fixed production network and fixed export participation is 
+ a wage level $w$,
+ a consumer price index $P$, and
+ aggregate expenditure $E$

such that equations $(5)$, $(7)-(9)$, and $(13)-(17)$ from TKMD (2018) hold. 

Recall that firms sell to final consumers at a *constant markup* over marginal cost but sell to other firms *at* marginal cost. Moreover, recall that the network is *acyclic*, so each firm can only use the output of its predecessors (and labor) to produce.

Because the network is fixed and all import/export decisions are taken as given here, **all fixed costs have been and will continue to be ignored.** 

In [15]:
# Calibrate elasticities and labor supply
sigma = 4.0
rho = 2.0
w = 1000.0 # normalized
tau = 1.0
mu = sigma / (sigma - 1)

# Arbitrarily choose foreign variable price vector
p_F = (rand(num,1) .+ 1.25)      # alternative: p_F = 1.75 * ones(num,1)
P_F = 0.05;      # alternative: P_F = (sum(p_F.^(1-sigma)))^(1/(sigma-1))

In [16]:
# Retrieve firm characteristic vectors
IndI = N[:,maxcard+3]
IndE = N[:,maxcard+4]
phi = N[:,maxcard+5]
X = N[:,maxcard+6]
Y = N[:,maxcard+7]
alpha_F = N[:,maxcard+8]
beta_F = N[:,maxcard+9];

In [17]:
c = zeros(num,1) 
Theta = zeros(num,1)
for j=1:num
    Z = N[j,3:maxcard+2]
    filter!(e -> e>0, Z)
    Z = Array{Int64}(Z)
    costsum = 0
    for i = Z
        temp = (X[i,1]*Y[j,1])^(rho-1)*(c[i,1])^(1-rho)
        costsum += temp
    end
    costsum += IndI[j,1]*(alpha_F[j,1])^(rho-1)*(p_F[j,1])^(1-rho)
    Theta[j,1] = costsum + w^(1-rho)
    c[j,1] = (1/phi[j,1])*(costsum + w^(1-rho))^(1/(1-rho)) # TKMD eq (7)
end
p_H = mu * c
P = (sum(p_H.^(1-sigma)))^(1/(sigma-1)); # TKMD eq (5)

In [18]:
@time begin
    S, SparseNet = Crews_SparseShares(N,c,Theta,p_F,rho)
    end;

  2.044059 seconds (100.05 k allocations: 106.023 MiB, 0.60% gc time)


In [19]:
E_EF = nlsolve((E_EF) -> Crews_FixNetEquil(E_EF, N, S, P, p_H, P_F, p_F, w, tau, sigma, rho)[1], ones(1,2))

Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: [1.0 1.0]
 * Zero: [7.70183e-41 2.93401e-30]
 * Inf-norm of residuals: 0.000000
 * Iterations: 7
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 8
 * Jacobian Calls (df/dx): 8

In [20]:
E_EF = E_EF.zero
E = E_EF[1]
E_F = E_EF[2];

In [21]:
MC_TB, L, total_cost, total_sale, total_profit, q, x_H, x_D, x_F = Crews_FixNetEquil(E_EF, N, S, P, p_H, P_F, p_F, w, tau, sigma, rho)

([-9.0001e-14, -9.00009e-25], 4.59684338873036e-34, [2.30565e-46; 2.70158e-39; … ; 1.93973e-37; 5.11273e-36], [3.0742e-46; 3.6021e-39; … ; 1.93973e-37; 5.11273e-36], [7.68551e-47; 9.00525e-40; … ; 1.55524e-47; 4.82367e-47], [1.44498e-9; 2.43532; … ; 1.7218e-10; 7.78796e-10], [3.06738e-46; 2.55146e-39; … ; 6.22099e-47; 1.92948e-46], 
  [2    ,     1]  =  5.46739e-60
  [3    ,     1]  =  1.30803e-58
  [4    ,     1]  =  5.15624e-60
  [5    ,     1]  =  4.78715e-58
  [6    ,     1]  =  2.84659e-57
  [7    ,     1]  =  7.17189e-59
  [8    ,     1]  =  4.1324e-59
  [9    ,     1]  =  8.42968e-58
  [10   ,     1]  =  6.71844e-60
  [11   ,     1]  =  4.54872e-57
  [12   ,     1]  =  4.58117e-59
  [13   ,     1]  =  2.93124e-58
  ⋮
  [9975 ,  9923]  =  1.11814e-39
  [9930 ,  9924]  =  3.04631e-50
  [9993 ,  9926]  =  1.53546e-40
  [9963 ,  9931]  =  2.41772e-43
  [9984 ,  9933]  =  2.10649e-39
  [9942 ,  9939]  =  2.79801e-39
  [9988 ,  9945]  =  1.72951e-48
  [9954 ,  9946]  =  3.04279e-47
  

The equilibrium expenditure values and labor endowment are ridiculously low, but at least they're all positive!

### Checking Moments

As already established, I am able to match the percentage of importers and exporters by construction. I hit the quantiles of the distribution of the number of suppliers quite well, but my distribution of labor shares is too spread out. I also overshoot the distribution of buyers. Later I show that my distribution of total foreign input shares is also a bit too spread out.

In [22]:
for p = [0.25, 0.5, 0.75]
    qShareLabor = quantile!(S[:, num+1], p)
    qNumSupply = quantile!(N[:,2], p)
    buyers = zeros(num,1)
    for j = 1:num
        buyers[j,1] = count(i -> i > 0, S[:,j])
    end
    qNumBuyers = quantile!(buyers[:,1], p)
    println("The p = ", p, " quantile of the distribution of labor cost shares is ", qShareLabor)
    println("The p = ", p, " quantile of the distribution of number of suppliers is ", qNumSupply)
    println("The p = ", p, " quantile of the distribution of number of buyers is ", qNumBuyers)
end

The p = 0.25 quantile of the distribution of labor cost shares is 0.08091384630879023
The p = 0.25 quantile of the distribution of number of suppliers is 20.0
The p = 0.25 quantile of the distribution of number of buyers is 13.0
The p = 0.5 quantile of the distribution of labor cost shares is 0.5810908730033146
The p = 0.5 quantile of the distribution of number of suppliers is 33.0
The p = 0.5 quantile of the distribution of number of buyers is 32.0
The p = 0.75 quantile of the distribution of labor cost shares is 0.9462935133491122
The p = 0.75 quantile of the distribution of number of suppliers is 56.0
The p = 0.75 quantile of the distribution of number of buyers is 63.0


### Calculating Foreign Input Shares

In [23]:
totFshare = zeros(num,1);
totFshare[1,1] = S[1,num+2];
for j = 2:num
    if N[j,2] == 0
        totFshare[j,1] = S[j,num+2]
    else
        Z = N[j,3:maxcard+2]
        filter!(e -> e>0, Z)
        Z = Array{Int64}(Z)
        totFshare[j,1] = S[j,num+2]
        for k = Z
            totFshare[j,1] += S[j,k]*S[k,num+2]
        end
    end
end

In [24]:
for p = [0.25, 0.5, 0.75]
    qtFs = quantile!(totFshare[:,1], p)
    println("The p = ", p, " quantile of the distribution of total foreign input shares is ", qtFs)
end

The p = 0.25 quantile of the distribution of total foreign input shares is 0.011689811775364904
The p = 0.5 quantile of the distribution of total foreign input shares is 0.190094067110629
The p = 0.75 quantile of the distribution of total foreign input shares is 0.7758889168801517


### Calculating the Real Wage Change from Reversion to Autarky 

Following the discussion below equation (20) of TKMD (2018), the change in the real wage from reversion to autarky is
$$
\Delta \left( \frac{w}{P} \right) = \left(\sum_j s_{jH} \left(1 - s_{Fj}^{Tot} \right)^{\frac{1-\sigma}{1-\rho}} \right)^{\frac{1}{\sigma-1}}.
$$
In my simulation the corresponding percentage change is 56%, which is not too far in magnitude from the estimate of 44% in section 3.3 of TKMD (2018).

In [25]:
ONE = ones(num,1)
sH = (p_H .^ (1-sigma)) / (P^(1-sigma))
dRealWage = (sum(sH[:,1] .* ((ONE[:,1]-totFshare[:,1]).^((1-sigma)/(1-rho))))^(1/(sigma-1)))*100 / w
dRealWage = ceil(dRealWage)
println("The percentage real wage change from reversion to autarky is roughly ", dRealWage, "%")

The percentage real wage change from reversion to autarky is roughly 11.0%


### The Effects of a 10% Foreign Price Increase

Consider a uniform 10% foreign price increase. We want to identify the concomitant change in the real wage. To do so, we used Propositions 1 and 2 as well as Section A.2 of the appendix of TKMD (2018).

In [42]:
# 10% increase in foreign price
p_F_hat = 1.1

# guess w_hat
x_hat = ones(num,1)
x_hat_previous = 1.1*ones(num,1)
w_hat = 1    # guess
w_hat_previous = 0.9;

share_H = p_H.*q
share_L = Array{Float64}(S[:, num+1]);

In [43]:
while abs(w_hat_previous-w_hat) > 10e-9
    w_hat_previous = w_hat
    c_hat = ((ones(num,1)-totFshare)*(w_hat^(1-rho)) + totFshare*(p_F_hat^(1-rho))).^(1/(1-rho))
    x_hat_F = c_hat.^(1-sigma) .* IndI
    P_hat = (c_hat'.^(1-sigma)*share_H)^(1/(1-sigma))
    share_L_hat = w_hat^(1-rho) * c_hat.^(rho-1)
    share_hat = c_hat'.^(1-rho).*SparseNet.*c_hat.^(1-rho)
    while norm(x_hat-x_hat_previous) > 10e-30
        x_hat_previous = x_hat
        E_hat = (w*L/E*w_hat) + (total_profit[:,1]'*x_hat_previous[:,1]/E)
        x_hat = x_F./total_sale.*x_hat_F + x_H./total_sale.*c_hat.^(1-sigma)*P_hat^(sigma-1)*E_hat + share_hat.*x_D*x_hat_previous./total_sale
        w_hat = (share_L .* share_L_hat)'*(total_cost .* x_hat/(w*L))
        w_hat = w_hat[1]
    end
end

In [44]:
real_wage_change = w_hat/P_hat[1]

6454.829290558672

Obviously **this result is outrageous**, but I haven't determined whether it is driven by a bug in the code or a flaw in the network formation or both.